In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

import geopandas as gpd

In [3]:
spark = SparkSession. \
    builder. \
    appName('BSSTD'). \
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.1.1-incubating,'
           'org.datasyslab:geotools-wrapper:1.1.0-25.2'). \
    getOrCreate()

C:\spark\spark-3.2.0-bin-hadoop3.2\python\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
SedonaRegistrator.registerAll(spark)

True

In [5]:
geodetskaTocka = gpd.read_file("C:/Users/ana00/Downloads/BSSTD/GeodetskaTocka.shp")
katCestica = gpd.read_file("C:/Users/ana00/Downloads/BSSTD/KatCestica.shp")
katOpcina = gpd.read_file("C:/Users/ana00/Downloads/BSSTD/KatOpcina.shp")
katPlan = gpd.read_file("C:/Users/ana00/Downloads/BSSTD/KatPlan.shp")
koristenjeZemljista = gpd.read_file("C:/Users/ana00/Downloads/BSSTD/KoristenjeZemljista.shp")
medjnaTocka = gpd.read_file("C:/Users/ana00/Downloads/BSSTD/MedjnaTocka.shp")
zgrada = gpd.read_file("C:/Users/ana00/Downloads/BSSTD/Zgrade.shp")

In [6]:
geodetskaTocka_geom = spark.createDataFrame(geodetskaTocka[["GID", "Broj", "Tip", "geometry"]])
geodetskaTocka_geom.createOrReplaceTempView("geodetskaTocka")

katCestica_geom = spark.createDataFrame(katCestica[["GID", "Broj", "KatOpcina", "GranicniKa", "PL", "Povrsina" ,"geometry"]])
katCestica_geom.createOrReplaceTempView("katCestica")

katOpcina_geom = spark.createDataFrame(katOpcina[["GID", "KO", "Naziv", "geometry"]])
katOpcina_geom.createOrReplaceTempView("katOpcina")

katPlan_geom = spark.createDataFrame(katPlan[["GID", "Broj", "geometry"]])
katPlan_geom.createOrReplaceTempView("katPlan")

koristenjeZemljista_geom = spark.createDataFrame(koristenjeZemljista[["GID", "KatCestica", "Tip", "Povrsina", "geometry"]])
koristenjeZemljista_geom.createOrReplaceTempView("koristenjeZemljista")

medjnaTocka_geom = spark.createDataFrame(medjnaTocka[["GID", "Broj", "Tip", "geometry"]])
medjnaTocka_geom.createOrReplaceTempView("medjnaTocka")

zgrada_geom = spark.createDataFrame(zgrada[["GID", "KatCestica", "Povrsina", "geometry"]])
zgrada_geom.createOrReplaceTempView("zgrada")

In [7]:
katOpcina_geom.show()

+---+------+------------+--------------------+
|GID|    KO|       Naziv|            geometry|
+---+------+------------+--------------------+
|1.0|4022.0|  Oeynhausen|POLYGON ((6372867...|
|2.0|4021.0| Möllersdorf|POLYGON ((6373674...|
|3.0|4038.0| Wienersdorf|POLYGON ((6372569...|
|4.0|4033.0|Traiskirchen|POLYGON ((6373821...|
|5.0|4034.0|Tribuswinkel|POLYGON ((6371317...|
+---+------+------------+--------------------+



In [8]:
q1 = spark.sql("""SELECT DISTINCT(katCestica.GID)
                    FROM katCestica 
                        JOIN geodetskaTocka 
                            ON ST_Within(geodetskaTocka.geometry, katCestica.geometry)
                    ORDER BY katCestica.GID""")
q1.show(q1.count(), False)

+------+
|GID   |
+------+
|6.0   |
|29.0  |
|44.0  |
|57.0  |
|106.0 |
|129.0 |
|134.0 |
|137.0 |
|147.0 |
|152.0 |
|155.0 |
|157.0 |
|198.0 |
|205.0 |
|222.0 |
|230.0 |
|232.0 |
|321.0 |
|347.0 |
|392.0 |
|435.0 |
|437.0 |
|448.0 |
|488.0 |
|534.0 |
|588.0 |
|636.0 |
|641.0 |
|643.0 |
|645.0 |
|651.0 |
|670.0 |
|695.0 |
|767.0 |
|790.0 |
|796.0 |
|808.0 |
|879.0 |
|907.0 |
|911.0 |
|913.0 |
|922.0 |
|971.0 |
|986.0 |
|994.0 |
|1045.0|
|1067.0|
|1090.0|
|1101.0|
|1107.0|
|1153.0|
|1228.0|
|1261.0|
|1264.0|
|1276.0|
|1281.0|
|1302.0|
|1367.0|
|1371.0|
|1391.0|
|1442.0|
|1478.0|
|1496.0|
|1552.0|
|1563.0|
|1613.0|
|1661.0|
|1716.0|
|1721.0|
|1763.0|
|1838.0|
|2019.0|
|2021.0|
|2041.0|
|2049.0|
|2070.0|
|2080.0|
|2142.0|
|2185.0|
|2317.0|
|2377.0|
|2379.0|
|2422.0|
|2463.0|
|2487.0|
|2541.0|
|2576.0|
|2584.0|
|2613.0|
|2631.0|
|2699.0|
|2739.0|
|2747.0|
|2750.0|
|2905.0|
|2932.0|
|2995.0|
|3087.0|
|3117.0|
|3118.0|
|3121.0|
|3146.0|
|3166.0|
|3196.0|
|3207.0|
|3271.0|
|3353.0|
|3379.0|
|

In [9]:
h2 = spark.sql("""SELECT * 
                    FROM katCestica
                    WHERE GranicniKa = 'G'
                    ORDER BY Povrsina DESC""").first()["GID"]
q2 = spark.sql("""SELECT katCestica.GID AS katCesticaGID, medjnaTocka.GID AS medjnaTockaGID
                    FROM medjnaTocka
                        JOIN katCestica
                            ON katCestica.GID = {}
                            AND ST_Intersects(katCestica.geometry, medjnaTocka.geometry)
                    ORDER BY medjnaTocka.GID""".format(h2))
q2.show(q2.count(), False)

+-------------+--------------+
|katCesticaGID|medjnaTockaGID|
+-------------+--------------+
|5506.0       |9506.0        |
|5506.0       |9507.0        |
|5506.0       |9715.0        |
|5506.0       |9876.0        |
|5506.0       |9902.0        |
|5506.0       |9921.0        |
|5506.0       |10313.0       |
|5506.0       |10316.0       |
|5506.0       |10319.0       |
|5506.0       |10395.0       |
|5506.0       |10396.0       |
|5506.0       |10397.0       |
|5506.0       |10398.0       |
|5506.0       |10399.0       |
|5506.0       |10400.0       |
|5506.0       |10401.0       |
|5506.0       |10402.0       |
|5506.0       |10403.0       |
|5506.0       |10404.0       |
|5506.0       |10406.0       |
|5506.0       |11089.0       |
|5506.0       |11093.0       |
|5506.0       |11100.0       |
|5506.0       |11117.0       |
|5506.0       |11142.0       |
|5506.0       |11169.0       |
|5506.0       |11994.0       |
|5506.0       |12013.0       |
|5506.0       |12026.0       |
|5506.0 

In [10]:
q3 = spark.sql("""SELECT DISTINCT(katCestica.GID)
                    FROM katCestica
                        JOIN koristenjeZemljista
                            ON koristenjeZemljista.Tip = 'Weingarten'
                            AND ST_Contains(koristenjeZemljista.geometry, katCestica.geometry)
                    ORDER BY katCestica.GID""")
q3.show(q3.count(), False)

+------+
|GID   |
+------+
|340.0 |
|344.0 |
|346.0 |
|358.0 |
|365.0 |
|373.0 |
|438.0 |
|458.0 |
|474.0 |
|475.0 |
|538.0 |
|574.0 |
|698.0 |
|700.0 |
|703.0 |
|742.0 |
|744.0 |
|785.0 |
|803.0 |
|859.0 |
|1713.0|
|1842.0|
|1864.0|
|1882.0|
|1890.0|
|1917.0|
|1919.0|
|1955.0|
|2005.0|
|2014.0|
|2017.0|
|2018.0|
|2050.0|
|2051.0|
|2100.0|
|2240.0|
|2306.0|
|2401.0|
|2420.0|
|2452.0|
|2475.0|
|2504.0|
|2505.0|
|2506.0|
|2507.0|
|2508.0|
|2509.0|
|2510.0|
|2511.0|
|2512.0|
|2513.0|
|2515.0|
|2517.0|
|2519.0|
|2520.0|
|2521.0|
|2522.0|
|2523.0|
|2525.0|
|2526.0|
|2527.0|
|2528.0|
|2529.0|
|2530.0|
|2543.0|
|2545.0|
|2546.0|
|2547.0|
|2548.0|
|2549.0|
|2551.0|
|2553.0|
|2554.0|
|2555.0|
|2556.0|
|2557.0|
|2558.0|
|2559.0|
|2560.0|
|2562.0|
|2563.0|
|2564.0|
|2565.0|
|2566.0|
|2567.0|
|2568.0|
|2569.0|
|2570.0|
|2571.0|
|2572.0|
|2574.0|
|2575.0|
|2577.0|
|2578.0|
|2579.0|
|2580.0|
|2581.0|
|2582.0|
|2583.0|
|2584.0|
|2585.0|
|2588.0|
|2589.0|
|2592.0|
|2595.0|
|2596.0|
|2679.0|
|2684.0|
|

In [11]:
h4 = spark.sql("""SELECT *
                    FROM katCestica
                        JOIN koristenjeZemljista
                            ON koristenjeZemljista.Tip = 'Weingarten'
                            AND ST_Contains(koristenjeZemljista.geometry, katCestica.geometry)
                    ORDER BY katCestica.Povrsina DESC""").first()["GID"]
q4 = spark.sql("""SELECT katCestica.GID as katCesticaGID, susjednaCestica.GID as susjednaCesticaGID
                    FROM katCestica AS susjednaCestica
                        JOIN katCestica
                            ON katCestica.GID = {}
                            AND ST_Touches(katCestica.geometry, susjednaCestica.geometry)
                    ORDER BY susjednaCestica.GID""".format(h4))
q4.show(q4.count(), False)

+-------------+------------------+
|katCesticaGID|susjednaCesticaGID|
+-------------+------------------+
|1842.0       |1803.0            |
|1842.0       |1826.0            |
|1842.0       |1838.0            |
|1842.0       |1842.0            |
|1842.0       |1864.0            |
|1842.0       |1890.0            |
|1842.0       |1976.0            |
+-------------+------------------+



In [12]:
q5 = spark.sql("""SELECT DISTINCT(katCestica.GID)
                    FROM katCestica
                        JOIN koristenjeZemljista
                            ON koristenjeZemljista.Tip = 'Landw. genutzt'
                            AND ST_Contains(koristenjeZemljista.geometry, katCestica.geometry)
                    ORDER BY katCestica.GID""")
q5.show(q5.count(), False)

+------+
|GID   |
+------+
|23.0  |
|27.0  |
|31.0  |
|36.0  |
|40.0  |
|43.0  |
|44.0  |
|45.0  |
|46.0  |
|47.0  |
|48.0  |
|50.0  |
|51.0  |
|53.0  |
|55.0  |
|56.0  |
|78.0  |
|104.0 |
|112.0 |
|114.0 |
|116.0 |
|118.0 |
|120.0 |
|121.0 |
|125.0 |
|126.0 |
|127.0 |
|128.0 |
|130.0 |
|131.0 |
|132.0 |
|133.0 |
|135.0 |
|136.0 |
|139.0 |
|140.0 |
|141.0 |
|142.0 |
|143.0 |
|144.0 |
|145.0 |
|146.0 |
|148.0 |
|149.0 |
|151.0 |
|152.0 |
|153.0 |
|154.0 |
|157.0 |
|159.0 |
|160.0 |
|161.0 |
|162.0 |
|163.0 |
|164.0 |
|196.0 |
|220.0 |
|222.0 |
|239.0 |
|244.0 |
|246.0 |
|247.0 |
|253.0 |
|254.0 |
|258.0 |
|259.0 |
|265.0 |
|270.0 |
|271.0 |
|273.0 |
|309.0 |
|315.0 |
|327.0 |
|332.0 |
|339.0 |
|341.0 |
|343.0 |
|351.0 |
|357.0 |
|359.0 |
|361.0 |
|364.0 |
|366.0 |
|396.0 |
|404.0 |
|407.0 |
|414.0 |
|427.0 |
|428.0 |
|431.0 |
|432.0 |
|446.0 |
|460.0 |
|468.0 |
|480.0 |
|488.0 |
|489.0 |
|503.0 |
|509.0 |
|515.0 |
|548.0 |
|553.0 |
|563.0 |
|594.0 |
|621.0 |
|653.0 |
|686.0 |
|688.0 |
|

In [13]:
q6 = spark.sql("""SELECT DISTINCT(katCestica.GID)
                    FROM katCestica
                        JOIN zgrada
                            ON ST_Contains(zgrada.geometry, katCestica.geometry)
                    ORDER BY katCestica.GID""")
q6.show(q6.count(), False)

+------+
|GID   |
+------+
|3.0   |
|10.0  |
|60.0  |
|65.0  |
|71.0  |
|74.0  |
|80.0  |
|84.0  |
|91.0  |
|93.0  |
|94.0  |
|99.0  |
|105.0 |
|170.0 |
|173.0 |
|174.0 |
|182.0 |
|183.0 |
|194.0 |
|201.0 |
|320.0 |
|386.0 |
|390.0 |
|587.0 |
|625.0 |
|631.0 |
|633.0 |
|642.0 |
|674.0 |
|718.0 |
|759.0 |
|771.0 |
|794.0 |
|850.0 |
|878.0 |
|886.0 |
|903.0 |
|950.0 |
|967.0 |
|972.0 |
|993.0 |
|996.0 |
|1001.0|
|1008.0|
|1010.0|
|1012.0|
|1014.0|
|1018.0|
|1020.0|
|1022.0|
|1025.0|
|1026.0|
|1031.0|
|1034.0|
|1039.0|
|1052.0|
|1066.0|
|1093.0|
|1094.0|
|1100.0|
|1102.0|
|1116.0|
|1135.0|
|1137.0|
|1163.0|
|1190.0|
|1207.0|
|1225.0|
|1243.0|
|1244.0|
|1266.0|
|1303.0|
|1304.0|
|1314.0|
|1318.0|
|1319.0|
|1324.0|
|1330.0|
|1331.0|
|1344.0|
|1349.0|
|1406.0|
|1408.0|
|1409.0|
|1430.0|
|1474.0|
|1486.0|
|1505.0|
|1506.0|
|1510.0|
|1527.0|
|1528.0|
|1541.0|
|1544.0|
|1560.0|
|1564.0|
|1565.0|
|1575.0|
|1589.0|
|1595.0|
|1604.0|
|1624.0|
|1654.0|
|1663.0|
|1679.0|
|1680.0|
|1689.0|
|1690.0|
|

In [14]:
q7 = spark.sql("""SELECT katCestica.GID
                    FROM katCestica
                        JOIN katPlan
                            ON katPlan.Broj = '7532G291'
                            AND ST_Contains(katPlan.geometry, katCestica.geometry)
                    ORDER BY katCestica.GID""")
q7.show(q7.count(), False)

+------+
|GID   |
+------+
|5731.0|
|5734.0|
|5736.0|
|5744.0|
|5751.0|
|6740.0|
|6742.0|
|6743.0|
|6744.0|
|6748.0|
|6888.0|
|6891.0|
|6893.0|
|6896.0|
|6897.0|
|6900.0|
|6902.0|
|6903.0|
|6905.0|
|6907.0|
|6910.0|
|6914.0|
|6915.0|
|6986.0|
|6987.0|
|6990.0|
|6992.0|
|6994.0|
|6999.0|
|7002.0|
|7003.0|
|7005.0|
|7010.0|
|7014.0|
|7015.0|
|7016.0|
|7021.0|
|7025.0|
|7031.0|
|7032.0|
|7033.0|
|7034.0|
|7035.0|
|7036.0|
|7037.0|
|7144.0|
|7148.0|
|7151.0|
|7152.0|
|7153.0|
|7155.0|
|7156.0|
|7158.0|
|7161.0|
|7163.0|
|7169.0|
|7171.0|
|7172.0|
|7173.0|
|7176.0|
|7183.0|
|7185.0|
|7188.0|
|7193.0|
|7194.0|
|7294.0|
|7298.0|
|7301.0|
|7303.0|
|7309.0|
|7310.0|
|7314.0|
|7318.0|
|7319.0|
|7322.0|
|7327.0|
|7331.0|
|7333.0|
|7442.0|
|7445.0|
|7448.0|
|7449.0|
|7450.0|
|7454.0|
|7455.0|
|7456.0|
|7458.0|
|7459.0|
|7460.0|
|7463.0|
|7466.0|
|7467.0|
|7468.0|
|7471.0|
|7473.0|
|7475.0|
|7477.0|
|7481.0|
|7482.0|
|7484.0|
|7485.0|
|7487.0|
|7489.0|
|7490.0|
|7491.0|
|7602.0|
|7604.0|
|7605.0|
|

In [15]:
q8 = spark.sql("""SELECT katOpcina.GID, katOpcina.Naziv, COUNT(*) AS brojGeodetskihTocaka
                    FROM katOpcina
                        JOIN geodetskaTocka
                            ON ST_Within(geodetskaTocka.geometry, katOpcina.geometry)
                    GROUP BY katOpcina.GID, katOpcina.Naziv
                    ORDER BY katOpcina.GID""")
q8.show()

+---+------------+--------------------+
|GID|       Naziv|brojGeodetskihTocaka|
+---+------------+--------------------+
|1.0|  Oeynhausen|                  30|
|2.0| Möllersdorf|                  71|
|3.0| Wienersdorf|                  59|
|4.0|Traiskirchen|                 130|
|5.0|Tribuswinkel|                  86|
+---+------------+--------------------+

